<div style="text-align: right; font-size: 0.8em; color: #666;">
最終更新: 2026-02-17 20:15 (Stable Model: llm-jp-3-3.7b)
</div>

# 05. LLMベースの軽量マルチエージェント（Executor/Critic）

このノートブックでは、LLM（3Bクラス）を使用して、**「実行者（Executor）」**と**「批評者（Critic）」**の2つの役割を連携させる最小構成のエージェントシステムを体験します。

### 本回でのモデル選択について
エージェントの演習では、安定した高速な推論が不可欠です。本回では、標準的なライブラリ構成で動作し、高速なキャッシュ生成に対応した **`llm-jp/llm-jp-3-3.7b-instruct`** を使用します。


In [ ]:
# 編集禁止セル
import os
import sys
import torch

repo_path = '/content/llm_lab'
# キャッシュ回避用タグ
UPDATE_TAG = '20260217_2015'

if os.path.exists(repo_path):
    !rm -rf {repo_path}

!git clone -b ai_agent https://github.com/akio-kobayashi/llm_lab.git {repo_path}
%cd {repo_path}

!pip install -q -U transformers accelerate bitsandbytes

if 'src' not in sys.path:
    sys.path.append(os.path.abspath('src'))

from src.common import load_llm
from src.agent_core import LLMExecutorCriticAgent

print(f'セットアップ完了 (Tag: {UPDATE_TAG})')


In [ ]:
# 編集禁止セル
# 第05回限定でモデルを差し替え（安定性と速度のため）
STABLE_MODEL_ID = "llm-jp/llm-jp-3-3.7b-instruct"
model, tokenizer = load_llm(model_id=STABLE_MODEL_ID, use_4bit=False)

def llm_chat(system_prompt: str, user_prompt: str, max_new_tokens: int = 512, temperature: float = 0.2):
    """標準的なモデル向けの高速・安定動作用チャット関数。"""
    try:
        # llm-jp-3 推奨のプロンプト形式
        prompt = (
            f"{system_prompt}\n\n"
            f"### 指示:\n{user_prompt}\n\n"
            "### 応答:\n"
        )

        inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=2048, truncation_side='left')
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        prompt_len = inputs['input_ids'].shape[1]

        # 標準的なモデルなので、特別なパッチなしでDynamicCacheが動作する
        with torch.inference_mode():
            gen_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=temperature,
                top_p=0.9,
                use_cache=True, # 高速な生成を有効化
                pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id,
                repetition_penalty=1.1,
            )

        new_ids = gen_ids[0][prompt_len:]
        text = tokenizer.decode(new_ids, skip_special_tokens=True).strip()

        return text if text else '[Empty output]'
    except Exception as e:
        return f'Error: {e}'

agent = LLMExecutorCriticAgent(llm_chat)

queries = [
    'Pythonで数値を2倍する関数の最小コードを書いて。',
    '2026年2月の最新アニメ情報を要約して。情報が足りない場合は前提を明示して。',
]

for q in queries:
    final_answer, full_log, steps = agent.run_pipeline(q)
    print('=' * 70)
    print('Q:', q)
    print('--- Full Pipeline Log ---')
    print(full_log)
    print('=' * 70)


## 演習

1. **プロンプトエンジニアリング**: `src/agent_core.py` 内の `Critic` の `system_prompt` を変更して、より厳しく、あるいはより優しく指摘するように調整してみましょう。
2. **温度（Temperature）の調整**: `RoleConfig` の `temperature` を変更して、Executor の回答の「創造性」と Critic の「正確性」がどう変化するか観察してください。
